- Introduction
- Data Load
- Data Preprocessing
- Visualization (Not done yet)
- Model
- Test
- Submit
    

## Introduction

While looking for the solution for this problem initially I was going towards NER way to solve it. But after reading different kernals and blog. I could figure out we can perform Q&A approch.

Clap for : This wonder full artical https://www.kaggle.com/jonathanbesomi/question-answering-starter-pack. 
Though I've made my on changes in training model.

### Approach

We have:

- Question: sentiment column (positive or negative), which I've converted into is positive, is negative or is neutral. Just to make it feel like a question. 
- Context: text column
- Answer: selected_text column


#### Note : 
- Please make sure while working on this problem your internet must be off. This is one of requirement on submittion our code base.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## Data Load

In [ ]:
train_df = pd.read_csv('/kaggle/input/tweet-sentiment-extraction/train.csv')
test_df = pd.read_csv('/kaggle/input/tweet-sentiment-extraction/test.csv')

In [ ]:
train_df.head()

In [ ]:
train_df.info()

### Category Distribution

Category disribution is fairly balanced. So one thing is sure in train data we should not borther about data imbalance.

In [ ]:
train_df['sentiment'].value_counts().plot(kind='bar')

Same here Category disribution is fairly balanced.

In [ ]:
test_df['sentiment'].value_counts().plot(kind='bar')

## Data pre-processing

We have almost no null rows / columns. We can drop them.

In [ ]:
train_df.count()

In [ ]:
train_df.dropna(inplace=True)

In [ ]:
train_df.count()

As we following Question Answer approach. We need to convert our train / test data into Q & A accepting format.

In [ ]:
def build_train_data(df):
    train_data = []
    for index in df.index:
        context = df.loc[index,'text']
        idx = df.loc[index,'textID']
        que = 'what is '+df.loc[index,'sentiment']
        ans = df.loc[index,'selected_text']
        start_pos = df.loc[index,'text'].find(df.loc[index,'selected_text'])
        
        qas = []
        answers = []
        ans_dict = {}
        qa_dict = {}
        ans_dict = {'text': ans,'answer_start':start_pos}
        answers.append(ans_dict)
        qa_dict = {'id':idx,'question': que,'is_impossible':False,'answers':answers}
        qas.append(qa_dict)
        cont_dict ={'context':context,'qas':qas}
        train_data.append(cont_dict)
    return train_data

In [ ]:
build_train_data(train_df)[0]

In [ ]:
test_df.head()

In [ ]:
def build_test_data(df):
    test_data = []
    for index in df.index:
        context = df.loc[index,'text']
        idx = df.loc[index,'textID']
        que = 'what is '+df.loc[index,'sentiment']

        
        qas = []
        answers = []
        ans_dict = {}
        qa_dict = {}
        ans_dict = {'text': '__NONE__','answer_start':1000000}
        answers.append(ans_dict)
        qa_dict = {'id':idx,'question': que,'is_impossible':False,'answers':answers}
        qas.append(qa_dict)
        cont_dict ={'context':context,'qas':qas}
        test_data.append(cont_dict)
    return test_data

In [ ]:
build_test_data(test_df)[0]

In [ ]:
distil_accept_train = build_train_data(train_df)

Here is sample converted data in Q & A format. 

In [ ]:
distil_accept_train[0:1]

In [ ]:
distil_accept_test = build_test_data(test_df)

In [ ]:
distil_accept_test[0:1]

export converted data into json is optional.

## Model Traninig

In [ ]:
!pip install '/kaggle/input/simple-transformers-pypi/seqeval-0.0.12-py3-none-any.whl' -q

In [ ]:
!pip install '/kaggle/input/simple-transformers-pypi/simpletransformers-0.22.1-py3-none-any.whl' -q

In [ ]:
from simpletransformers.question_answering import QuestionAnsweringModel

This is pre-trained model (DistilBert + Squad). You can download it for offline use.
https://www.kaggle.com/jonathanbesomi/transformers-pretrained-distilbert.

In [ ]:
model_path = '/kaggle/input/transformers-pretrained-distilbert/distilbert-base-uncased-distilled-squad'

In [ ]:
%%time

model = QuestionAnsweringModel('distilbert',model_path,args={'reprocess_input_data': True,
                                     'overwrite_output_dir': True,                    
                                     'learning_rate': 5e-5,
                                     'num_train_epochs': 3,
                                     'max_seq_length': 192,
                                     'doc_stride': 64,
                                     'fp16': False,                      
                                    },use_cuda=True)

In [ ]:
%%time
model.train_model(distil_accept_train)

In [ ]:
pred =  model.predict(distil_accept_test)

In [ ]:
pred[0:2]

In [ ]:
sub_df = pd.read_csv('/kaggle/input/tweet-sentiment-extraction/sample_submission.csv')

In [ ]:
submit_df = pd.DataFrame.from_dict(pred)
sub_df['selected_text'] = submit_df['answer']
sub_df.to_csv('submission.csv',index=False)
print('successfully submit')

In [ ]:
import shutil
# import os

In [ ]:
#os.remove('train.json')

In [ ]:
#shutil.rmtree('cache_dir/')